### Análisis de Datos del Mercado de Criptomonedas con Binance API

- En la era digital actual, el mercado de criptomonedas ha emergido como un campo fascinante y dinámico, ofreciendo oportunidades únicas para el análisis de datos y la comprensión de las tendencias del mercado financiero. Nuestro proyecto se centra en explorar este sector vibrante utilizando una de las herramientas más poderosas en el mundo de las criptomonedas: la API de Binance.

- Binance, siendo una de las plataformas de intercambio de criptomonedas más grandes y reconocidas a nivel mundial, proporciona una API rica y robusta, permitiendo a los usuarios acceder a una amplia gama de datos del mercado en tiempo real. Esto incluye, pero no se limita a, datos históricos de precios, detalles de operaciones individuales, volúmenes de trading, y mucho más.

- El objetivo principal de este proyecto es aprovechar estos datos para obtener insights valiosos sobre el comportamiento del mercado de Bitcoin (BTC), la criptomoneda líder en el mercado. Nos centraremos en extraer datos históricos de precios y operaciones de BTC comparados con el USD, analizando las tendencias, volatilidades y posibles patrones de trading.

- Al final de este proyecto, esperamos tener un análisis detallado y multifacético del mercado de Bitcoin, brindando un panorama claro no solo para los entusiastas de las criptomonedas, sino también para aquellos interesados en las finanzas y la tecnología. Este desafío es una excelente oportunidad para sumergirnos en el mundo del análisis de datos financieros y abrir la puerta a futuras investigaciones y proyectos en este apasionante campo.

# Introduccion

En el presente proyecto se prevee analizar las variaciones de bitcoin, Intentando agregar valores foraneos como comentarios en redes o fragmentos de noticias recabados de internet.

La finalidad del mismo es tratar de predecir a corto plazo el valor de la moneda, De esta forma crear un bot de trading de alto flujo y que pueda ser porbado en un entorno de produccion

Los bot de trading siempre se utilizaron con algorimos basicos y ultimamente estan utilizando data Science para mejorar sus modelos, Lo que presenta un mercado en crecimiento y un nicho de mercado importante


# Descripcion

Para esto utilizaremos la API de binance a travez de su cliente en python, La cual nos deja acceso a muchas variables pero en estos momentos solo nos centraremos en el precio. 

Una vez el modelo este en funcionamiento, la misma api de binance permite realizar compras y ventas de BTC en un muy corto periodo de tiempo

In [1]:
# paquetes necesarios
import requests
import pandas as pd
import hmac
import hashlib
import time
from datetime import datetime, timedelta, timezone
import matplotlib.pyplot as plt
from decouple import config
from binance.client import Client
api_key = config('API_KEY')
api_secret = config('API_SECRET')
api_limit = 1000
requestDays = 3365
limit = 1000

## Descargar ultimos precios

In [2]:
# Crear una instancia del cliente de Binance
client = Client(api_key, api_secret)

# Calcular las fechas para el último mes
end_date = datetime.now(timezone.utc)
start_date = end_date - timedelta(days=requestDays)

# Convertir las fechas a milisegundos (que es el formato que la API de Binance espera)
start_date_timestamp = int(start_date.timestamp() * 1000)
end_date_timestamp = int(end_date.timestamp() * 1000)

# Obtener los datos del precio del Bitcoin para el último mes con intervalo de 1 minuto
symbol = 'BTCUSDT'
interval = Client.KLINE_INTERVAL_1MINUTE

In [3]:

# Inicializar una lista para almacenar los datos
all_klines = []

# Hacer llamadas sucesivas para obtener más datos (limitando cada llamada a 500)

while start_date_timestamp < end_date_timestamp:
    klines = client.get_klines(symbol=symbol, interval=interval, startTime=start_date_timestamp, endTime=end_date_timestamp, limit=limit)
    
    # Verificar si hay datos en la respuesta
    if not klines:
        break
    
    all_klines.extend(klines)
    
    # Actualizar el startTime para la siguiente llamada
    start_date_timestamp = int((pd.to_datetime(klines[-1][0], unit='ms') + timedelta(minutes=1)).timestamp() * 1000)

# Comprobar si se recuperaron datos antes de crear el DataFrame
if all_klines:
    # Crear un DataFrame de Pandas con todos los datos
    columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']
    df = pd.DataFrame(all_klines, columns=columns)

    # Convertir el timestamp a formato de fecha y hora legible
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

    # Mostrar el DataFrame
    df.info()
else:
    print("No se encontraron datos para el período especificado.")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3442256 entries, 0 to 3442255
Data columns (total 12 columns):
 #   Column                        Dtype         
---  ------                        -----         
 0   timestamp                     datetime64[ns]
 1   open                          object        
 2   high                          object        
 3   low                           object        
 4   close                         object        
 5   volume                        object        
 6   close_time                    int64         
 7   quote_asset_volume            object        
 8   number_of_trades              int64         
 9   taker_buy_base_asset_volume   object        
 10  taker_buy_quote_asset_volume  object        
 11  ignore                        object        
dtypes: datetime64[ns](1), int64(2), object(9)
memory usage: 315.1+ MB


In [4]:
df.sample(5)

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
372957,2018-05-05 01:26:00,9749.72000000,9751.92000000,9740.09000000,9741.34000000,7.93949400,1525483619999,77385.51682704,96,5.35626800,52219.90382471,0
377888,2018-05-08 11:37:00,9234.99000000,9239.48000000,9232.38000000,9236.77000000,16.21374000,1525779479999,149751.95779818,149,5.76894400,53287.96336271,0
1619878,2020-09-20 01:45:00,10971.07000000,10978.42000000,10969.35000000,10971.09000000,44.29018300,1600566359999,486001.77276704,463,34.93984500,383401.53815164,0
1640778,2020-10-04 14:05:00,10612.24000000,10612.25000000,10608.09000000,10609.80000000,24.76967600,1601820359999,262797.67356599,279,16.04159400,170191.64540433,0
3231165,2023-10-15 00:37:00,26821.89000000,26821.89000000,26815.76000000,26816.98000000,11.56553000,1697330279999,310181.03280590,751,2.68387000,71972.63663920,0


In [5]:
## save df to backup.json
df.to_json('backup.json')

In [ ]:
## recuperar backup.json to df
df = pd.read_json('backup.json')
df.info()